# Q-Learning

## Imports

In [14]:
import random
from collections import defaultdict, namedtuple
from itertools import product, starmap

import gym
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from scipy import stats
%matplotlib inline
sns.set()
states_colors = matplotlib.colors.ListedColormap(['#9A9A9A', '#D886BA', '#4D314A', '#6E9183'])
cmap_default = 'Blues'
cpal_default = sns.color_palette(('Blues_d'))

sns.set_style('white')
sns.set_context('poster')
random.seed(1)

## Grid World 4x12

The Agent has to deal with an environment consisting of 4 x 12 squares, representing all of 
its possible states. The mechanics of this world are defined bya small set of rules:

1. 

Implement Q-Table learning algorithm

In [16]:
# Initialize table with all zeros
Q = np.zeros([env.observation_space.n, env.action_space.n])

# set learning parameters
lr = .8
y = .95
num_episodes = 2000

# create list to contain total rewards and steps por episode
rList = []

for i in range(num_episodes):
    # reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    
    # the Q-table learning algorithm
    while j < 99:
        j += 1
        # choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s, :] + np.random.randn(1, env.action_space.n) * (1./(i+1)))
        # get new state and reward from environment
        s1, r, d, _ = env.step(a)
        # update q-table with new knowledge
        Q[s, a] = Q[s, a] + lr * (r + y * np.max(Q[s1,:]) - Q[s, a])
        rAll += r
        s = s1
        if d:
            break
    rList.append(rAll)
    

In [17]:
print('Score over time: ', str(sum(rList)/num_episodes))


Score over time:  0.463


In [18]:
print('Final Q-Table Values', Q)

Final Q-Table Values [[9.77705378e-02 8.05737313e-03 7.92383733e-03 8.27396692e-03]
 [2.58483388e-03 3.68103495e-04 1.26244933e-03 7.25962326e-02]
 [1.19547998e-01 2.53210566e-03 3.34234510e-03 4.59599583e-03]
 [4.84440713e-04 1.81243732e-03 1.23000740e-03 5.07344342e-03]
 [2.37452714e-01 2.20345406e-03 1.79642482e-04 2.03113243e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.33852461e-02 2.13114690e-05 4.44523970e-04 1.89780326e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.51184619e-04 1.62865462e-03 1.91082407e-03 4.40775031e-01]
 [7.51907316e-04 2.34704437e-01 1.30922531e-03 0.00000000e+00]
 [1.01648858e-01 0.00000000e+00 6.72713559e-04 2.87463787e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.27722758e-03 1.28132313e-03 3.89777075e-01 4.17534533e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.93853927e-01]
 [0.00000000e+00 0.00000000e+00 0.